# Algoritmos de Optimización.
## Samuel De Dios

## Programación lineal

Importar libreria.

In [7]:
!pip install pulp
from pulp import *


[notice] A new release of pip is available: 23.3.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Se usa de ejemplo este ejercicio.

\begin{align*}
\text{Minimizar }&19x_{1}+57x_{2} \\
\text{Restric }& x_{1} + 2x_{2} \geq 10\\
& x_{1} + 4x_{2} \geq 24\\
& 3x_{1} + 2x_{2} \geq 20\\
& 5x_{1} + x_{2} \geq 25\\
\end{align*}

Se define un objeto problema, luego se pone nombre y lo que se quiere hacer (minimizar o maximizar), luego se definen las variables del problema.

In [8]:
problema = LpProblem("proLineal", LpMinimize)
x1 = LpVariable("x1", lowBound = 0)
x2 = LpVariable("x2", lowBound = 0)

Se agrega primero la función objetivo y despues las restricciones.

In [9]:
problema += 19*x1 + 57*x2
problema += x1 + 2*x2 >= 10
problema += x1 + 4*x2 >= 24
problema += 3*x1 + 2*x2 >= 20
problema += 5*x1 + x2 >= 25

Se solucciona el problema.

In [10]:
problema.solve()

print(f"x1 = {x1.varValue} y x2 = {x2.varValue}, con valor optimo {value(problema.objective)}")

x1 = 4.0 y x2 = 5.0, con valor optimo 361.0
